<a href="https://colab.research.google.com/github/deeponcology/PyTorchMedicalAI/blob/master/shlomo_dl_0009_tensor_rt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning in Medical AI 2018/2019 using PyTorch & Google Collab.


<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/drive/1hGVoskPSerJtUydCoKeUskQG8X4lt3k9?authuser=1#scrollTo=bTPWD8_Bde6e">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/deeponcology/PyTorchMedicalAI/blob/master/shlomo_dl_0009_tensor_rt.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

<img src="https://github.com/deeponcology/PyTorchMedicalAI/raw/master/assets/tumor_visdom.jpg" align="center" width=30%>

### Author: 
***Shlomo Kashani***, Head of AI at www.DeepOncology.AI, shlomo@deeponcology.ai 


### Synopsys:
Using TensorRT to extract VGG features using the paper:https://github.com/alexander-rakhlin/ICIAR2018 

<img src="https://github.com/alexander-rakhlin/ICIAR2018/raw/master/pics/nn_diagram.png" align="center" width=50%>

### DataSets:
https://github.com/alexander-rakhlin/ICIAR2018


# Collab notebook: 009 using TensorRT with Keras and TF

In [0]:
%reset -f

In [0]:
! wget https://gist.githubusercontent.com/ashokpant/5c4e9481615f54af4025ab2085f85869/raw/e09f10a30ce1b40018f4440df9de788f5d6fdad0/cuda_9.0_cudnn_7.0.sh
! chmod +x cuda_9.0_cudnn_7.0.sh
! sudo bash cuda_9.0_cudnn_7.0.sh

--2019-02-02 20:38:00--  https://gist.githubusercontent.com/ashokpant/5c4e9481615f54af4025ab2085f85869/raw/e09f10a30ce1b40018f4440df9de788f5d6fdad0/cuda_9.0_cudnn_7.0.sh
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1317 (1.3K) [text/plain]
Saving to: ‘cuda_9.0_cudnn_7.0.sh’

cuda_9.0_cudnn_7.0. 100%[===================>]   1.29K  --.-KB/s    in 0s      

2019-02-02 20:38:00 (224 MB/s) - ‘cuda_9.0_cudnn_7.0.sh’ saved [1317/1317]

--2019-02-02 20:38:02--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awai

In [0]:
! sudo apt-get --fix-broken install
! nvcc --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  libcudnn7 libcudnn7-dev
The following packages will be upgraded:
  libcudnn7 libcudnn7-dev
2 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Need to get 285 MB of archives.
After this operation, 96.3 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  libcudnn7-dev 7.4.2.24-1+cuda10.0 [136 MB]
Get:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  libcudnn7 7.4.2.24-1+cuda10.0 [149 MB]
Fetched 285 MB in 6s (45.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline


# TRT version
We need to download this file and access it from Drive
- https://developer.nvidia.com/nvidia-tensorrt-4x-download
- https://developer.nvidia.com/compute/machine-learning/tensorrt/4.0/ga/TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.2.cudnn7.1

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
! pwd
! ls -la '/content/drive/My Drive/TRT/'

# trt_path='/content/drive/My Drive/TRT/TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.2.cudnn7.1.tar.gz'


/content
total 798520
-rw------- 1 root root     17854 Feb  2 18:11 image227.png
-rw------- 1 root root 410872039 Feb  2 14:04 TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.0.cudnn7.1.tar.gz
-rw------- 1 root root 406793742 Feb  2 12:37 TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.2.cudnn7.1.tar.gz


# References:
- https://docs.nvidia.com/deeplearning/sdk/tensorrt-install-guide/index.html
- https://tsmatz.wordpress.com/2018/07/07/tensorrt-tensorflow-python-on-azure-tutorial/
- https://jkjung-avt.github.io/tf-trt-models/
- https://kezunlin.me/post/dacc4196/
- https://blog.csdn.net/MacwinWin/article/details/80177326


# Install TRT form the TAR file
- https://kezunlin.me/post/dacc4196/

In [7]:
import os
os.chdir('/content/drive/My Drive/TRT/')
! tar zxvf 'TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.0.cudnn7.1.tar.gz'

TensorRT-4.0.1.6/
TensorRT-4.0.1.6/data/
TensorRT-4.0.1.6/data/mnist/
TensorRT-4.0.1.6/data/mnist/5.pgm
TensorRT-4.0.1.6/data/mnist/mnist.caffemodel
TensorRT-4.0.1.6/data/mnist/deploy.prototxt
TensorRT-4.0.1.6/data/mnist/mnist_lenet.caffemodel
TensorRT-4.0.1.6/data/mnist/lenet5_custom_pool.uff
TensorRT-4.0.1.6/data/mnist/4.pgm
TensorRT-4.0.1.6/data/mnist/batches/
TensorRT-4.0.1.6/data/mnist/batches/batch851
TensorRT-4.0.1.6/data/mnist/batches/batch690
TensorRT-4.0.1.6/data/mnist/batches/batch396
TensorRT-4.0.1.6/data/mnist/batches/batch321
TensorRT-4.0.1.6/data/mnist/batches/batch964
TensorRT-4.0.1.6/data/mnist/batches/batch640
TensorRT-4.0.1.6/data/mnist/batches/batch254
TensorRT-4.0.1.6/data/mnist/batches/batch925
TensorRT-4.0.1.6/data/mnist/batches/batch756
TensorRT-4.0.1.6/data/mnist/batches/batch186
TensorRT-4.0.1.6/data/mnist/batches/batch928
TensorRT-4.0.1.6/data/mnist/batches/batch884
TensorRT-4.0.1.6/data/mnist/batches/batch980
TensorRT-4.0.1.6/data/mnist/batches/batch759
Tens

In [0]:
! sudo rm -rf /opt/TensorRT-4.0.1.6/
! sudo mv TensorRT-4.0.1.6 /opt/

In [9]:
! ls -la '/opt/TensorRT-4.0.1.6/'

total 1564
drwx------ 9 root root    4096 Jun 12  2018 .
drwxr-xr-x 1 root root    4096 Feb  2 20:46 ..
drwx------ 8 root root    4096 Feb  2 20:46 data
drwx------ 5 root root    4096 Jun 12  2018 doc
drwx------ 2 root root    4096 Jun 12  2018 graphsurgeon
drwx------ 2 root root    4096 Jun 12  2018 include
drwx------ 4 root root    4096 Jun 12  2018 python
drwx------ 3 root root    4096 Jun 12  2018 targets
-r-------- 1 root root 1563257 Jun 12  2018 TensorRT-Release-Notes.pdf
drwx------ 2 root root    4096 Jun 12  2018 uff


In [0]:
import os 
os.chdir('/opt/TensorRT-4.0.1.6/')
! sudo ln -s TensorRT-4.0.1.6/ tensorrt


# TRT Python 

In [11]:
os.chdir('/opt/TensorRT-4.0.1.6/python/')
! ls -la 
!sudo pip3 install tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl 

total 3096
drwx------ 4 root root    4096 Jun 12  2018 .
drwx------ 9 root root    4096 Feb  2 20:47 ..
drwx------ 3 root root    4096 Jun 12  2018 data
drwx------ 4 root root    4096 Jun 12  2018 doc
-rw------- 1 root root 1571581 Jun 12  2018 tensorrt-4.0.1.6-cp27-cp27mu-linux_x86_64.whl
-rw------- 1 root root 1579848 Jun 12  2018 tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl
tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl is not a supported wheel on this platform.


In [0]:
! mv tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl tensorrt-4.0.1.6-cp36-cp36m-linux_x86_64.whl

In [8]:
!sudo pip3 install tensorrt-4.0.1.6-cp36-cp36m-linux_x86_64.whl 

Requirement 'tensorrt-4.0.1.6-cp36-cp36m-linux_x86_64.whl' looks like a filename, but the file does not exist


In [9]:
os.chdir('/opt/TensorRT-4.0.1.6/uff')
! sudo pip3 install uff-0.4.0-py2.py3-none-any.whl 


In [10]:
!which convert-to-uff

/usr/local/bin/convert-to-uff


In [11]:
! ls -la /opt/TensorRT-4.0.1.6/graphsurgeon/
os.chdir('/opt/TensorRT-4.0.1.6/graphsurgeon')
! sudo pip3 install graphsurgeon-0.2.0-py2.py3-none-any.whl

total 24
drwx------ 2 root root  4096 Jun 12  2018 .
drwx------ 9 root root  4096 Feb  2 20:47 ..
-rw------- 1 root root 13338 Jun 12  2018 graphsurgeon-0.2.0-py2.py3-none-any.whl


In [12]:
! ls -la /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/
! ls -la /usr/local/cuda


total 20
drwx------  5 root root 4096 Jun 12  2018 .
drwx------  3 root root 4096 Jun 12  2018 ..
drwx------  2 root root 4096 Jun 12  2018 bin
drwx------  2 root root 4096 Feb  2 20:49 lib
drwx------ 17 root root 4096 Jun 12  2018 samples
lrwxrwxrwx 1 root root 8 Feb  2 20:40 /usr/local/cuda -> cuda-9.0


In [13]:
ls -la /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/

total 20
drwx------  5 root root 4096 Jun 12  2018 ./
drwx------  3 root root 4096 Jun 12  2018 ../
drwx------  2 root root 4096 Jun 12  2018 bin/
drwx------  2 root root 4096 Feb  2 20:49 lib/
drwx------ 17 root root 4096 Jun 12  2018 samples/


In [0]:
os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/" # NOTE NO + SIGN 
os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['LD_LIBRARY_PATH']+= ":/usr/lib64-nvidia/:/opt/TensorRT-4.0.1.6/include/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/"
os.environ['PATH']+= ":/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/"


In [21]:

# !env | grep cuda
!env | grep LD_LIBRARY_PATH
# /opt/TensorRT-4.0.1.6/include/

LD_LIBRARY_PATH=/usr/lib64-nvidia/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib//opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/:/usr/lib64-nvidia/:/opt/TensorRT-4.0.1.6/include/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin//opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/:/usr/lib64-nvidia/:/opt/TensorRT-4.0.1.6/include/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/


In [22]:
# ! ls -la  /opt/TensorRT-4.0.1.6/include
%%file /etc/ld.so.conf.d/tensorrt

#/opt/TensorRT-4.0.1.6/lib
/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib


Overwriting /etc/ld.so.conf.d/tensorrt


In [23]:
! sudo ldconfig

! sudo cp /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib* /usr/lib
! sudo cp /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin* /usr/bin
! sudo cp /opt/TensorRT-4.0.1.6/include/* /usr/local/include/

cp: -r not specified; omitting directory '/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib'
cp: -r not specified; omitting directory '/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin'


In [0]:
os.chdir('/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/samples/sampleMNIST')

In [0]:
#! make -j 8
# Makefile  sampleMNIST.cpp

#os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/" # NOTE NO + SIGN 
#os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
#os.environ['LD_LIBRARY_PATH']= "/opt/TensorRT-4.0.1.6/include/"



In [25]:
# see https://stackoverflow.com/questions/50621536/tensorrt-python-package-incompatibility-with-python-3-6/50793199 
! sudo find / -name *nv_infer_bindings.cpython-35m-x86_64-linux-gnu.so

find: ‘/proc/2718/task/2718/net’: Invalid argument
find: ‘/proc/2718/net’: Invalid argument


In [26]:
! sudo mv /usr/local/lib/python3.6/dist-packages/tensorrt/infer/_nv_infer_bindings.cpython-35m-x86_64-linux-gnu.so /usr/local/lib/python3.6/dist-packages/tensorrt/infer/_nv_infer_bindings.cpython-36m-x86_64-linux-gnu.so

mv: cannot stat '/usr/local/lib/python3.6/dist-packages/tensorrt/infer/_nv_infer_bindings.cpython-35m-x86_64-linux-gnu.so': No such file or directory


In [26]:
! wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb
! wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.3.0.29-1+cuda9.0_amd64.deb  

--2019-02-02 20:47:56--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.211.70, 2606:2800:21f:3aa:dcf:37b:1ed6:1fb
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|192.229.211.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122730426 (117M) [application/x-deb]
Saving to: ‘libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb’

libcudnn7_7.2.1.38- 100%[===================>] 117.04M  3.33MB/s    in 35s     

2019-02-02 20:48:31 (3.30 MB/s) - ‘libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb’ saved [122730426/122730426]

--2019-02-02 20:48:32--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.3.0.29-1+cuda9.0_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.211.70, 2606:2800:21f:3aa:dcf:37b:1ed6:1fb
Connec

In [1]:
 ! sudo find / -name tensorflow*



/usr/local/lib/python3.6/dist-packages/uff/converters/tensorflow
/usr/local/lib/python3.6/dist-packages/vis/backend/tensorflow_backend.py
/usr/local/lib/python3.6/dist-packages/vis/backend/__pycache__/tensorflow_backend.cpython-36.pyc
/usr/local/lib/python3.6/dist-packages/imblearn/tensorflow
/usr/local/lib/python2.7/dist-packages/pyarrow/include/arrow/adapters/tensorflow
/usr/local/lib/python2.7/dist-packages/pyarrow/tensorflow
/usr/local/lib/python2.7/dist-packages/vis/backend/tensorflow_backend.pyc
/usr/local/lib/python2.7/dist-packages/vis/backend/tensorflow_backend.py
/usr/local/lib/python2.7/dist-packages/imblearn/tensorflow
/usr/local/lib/python2.7/dist-packages/tensorboard/_vendor/tensorflow_serving
/usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.pyc
/usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py
find: ‘/proc/2718/task/2718/net’: Invalid argument
find: ‘/proc/2718/net’: Invalid argument


In [6]:
#! sudo dpkg -i libcudnn7_7.3.0.29-1+cuda9.0_amd64.deb # 7.4.2.24-1+cuda10.0 to 7.3.0.29-1+cuda9.0
! sudo rm -rf /usr/local/lib/python3.6/dist-packages/tensorflow*
! sudo rm -rf /usr/local/lib/python3.6/dist-packages/keras*
! sudo rm -rf  /usr/local/lib/python3.6/dist-packages/keras/
! sudo rm -rf  /usr/local/lib/python2.7/dist-packages/tensorflow*
! sudo pip3 uninstall tensorflow tensorflow-gpu protobuf keras tensorboard
! sudo pip3 install tensorflow-gpu==1.8 keras
#! sudo pip3 install --upgrade tensorflow-gpu



Skipping tensorflow as it is not installed.
Skipping tensorflow-gpu as it is not installed.
Skipping protobuf as it is not installed.
Skipping keras as it is not installed.
Skipping tensorboard as it is not installed.
    100% |████████████████████████████████| 216.3MB 107kB/s 
    100% |████████████████████████████████| 317kB 28.6MB/s 
    100% |████████████████████████████████| 3.1MB 11.0MB/s 
    100% |████████████████████████████████| 1.1MB 17.0MB/s 
    100% |████████████████████████████████| 890kB 24.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
tfds-nightly 0.0.2.dev201901300014 requires tensorflow-metadata, which is not installed.
tensor2tensor 1.11.0 requires tensorflow-probability, which is not installed.
fancyimpute 0.4.2 requires tensorflow, which is not installed.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5l

In [28]:

#! sudo find / -name libnvinfer.so.* 
! sudo ln -s /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/libnvinfer.so.4.1.2 /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/libnvinfer.so.4

! sudo cp -r /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/*.* /usr/lib/ # SUPER CRITICAL 
! sudo cp -r /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/*.* /usr/bin/ # SUPER CRITICAL 

#/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/

import os
os.environ['LD_LIBRARY_PATH']+= "/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/"

! python -c "import tensorflow.contrib.tensorrt as trt; print(trt.__path__)"


from tensorflow.contrib import tensorrt as trt
import tensorflow 
print (tensorflow.__version__)

ln: failed to create symbolic link '/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/libnvinfer.so.4': File exists
['/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tensorrt']
1.8.0


# VGG in Keras

In [30]:
! sudo  pip3 install keras_applications==1.0.4 --no-deps
! sudo pip3 install keras_preprocessing==1.0.2 --no-deps

import os
import zipfile
import hashlib
from six.moves.urllib.error import URLError
from six.moves.urllib.request import urlretrieve
import argparse
from tqdm import tqdm
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Concatenate
import numpy as np

import numpy as np
import pandas as pd
import time, os, glob
import cv2

from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD, Adam
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input

from os.path import isfile, join, abspath, exists, isdir, expanduser
from os import listdir, makedirs, getcwd, remove

    100% |████████████████████████████████| 51kB 2.5MB/s 
  Found existing installation: Keras-Applications 1.0.7
    Uninstalling Keras-Applications-1.0.7:
      Successfully uninstalled Keras-Applications-1.0.7
  Found existing installation: Keras-Preprocessing 1.0.8
    Uninstalling Keras-Preprocessing-1.0.8:
      Successfully uninstalled Keras-Preprocessing-1.0.8


## Define the VGG feature extractor

In [31]:
from tqdm import tqdm # Enable progress bar
from keras.models import Model
from keras.preprocessing import image
from keras.layers import Flatten, Input

from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, warnings
    
class VGGFV:
    __name__ = "VGGFV"

    def __init__(self, feature_layer="block5_conv3", input_size=224,num_class=1,batch_size=32):
        base_model = VGG16(weights='imagenet', include_top=False,
                           input_shape=[input_size,input_size,3], classes=num_class)
                                        
        x2 = GlobalAveragePooling2D()(base_model.get_layer("block2_conv2").output)  # 128
        x3 = GlobalAveragePooling2D()(base_model.get_layer("block3_conv3").output)  # 256
        x4 = GlobalAveragePooling2D()(base_model.get_layer("block4_conv3").output)  # 512
        x5 = GlobalAveragePooling2D()(base_model.get_layer("block5_conv3").output)  # 512        
        x = Concatenate(name="concat_1408")([x2,x3,x4,x5])  
#         x= concatenate([x2,x3,x4,x5], name="concat_1408")

        model = Model(inputs=base_model.input, outputs=x)        
#         optimizer = Adam(lr=0.0001)
#         model.compile(loss='categorical_crossentropy',
#                       optimizer=optimizer,
#                       metrics=['accuracy'])
        
        self.model=model    
        self.batch_size = batch_size
        self.data_format = K.image_data_format()
    
    def predict(self, x):
        if self.data_format == "channels_first":
            x = x.transpose(0, 3, 1, 2)
        x = preprocess_vgg(x.astype(K.floatx()))
        return self.model.predict(x, batch_size=self.batch_size).squeeze()
    

img_path = "/content/drive/My Drive/TRT/image227.png"
feature_layer = "block5_conv3" # 512
input_size=224

img = image.load_img(img_path, target_size=(input_size, input_size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_vgg(x)
print (x.shape)

model = VGGFV (feature_layer, input_size=input_size,num_class=1)
model.model.summary()


vgg_feature_vector = model.predict(x)
print (vgg_feature_vector.shape)
vgg_feature_vector    

(1, 224, 224, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

array([3.2082626e+02, 1.5750623e+02, 1.3263094e+02, ..., 0.0000000e+00,
       6.1613008e-02, 1.8149542e-02], dtype=float32)

In [33]:
%timeit model.predict(x)
%timeit model.predict(x)
%timeit model.predict(x)

10 loops, best of 3: 23.8 ms per loop
10 loops, best of 3: 23.6 ms per loop
10 loops, best of 3: 23.6 ms per loop


# Tf frozen VGG graph

In [34]:
import tensorflow as tf 
sess = K.get_session()
output_names = ['concat_1408/concat'] # concat_1408, TF changes teh names depending on the SESSION

frozen_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), output_names)
frozen_graph = tf.graph_util.remove_training_nodes(frozen_graph)
# Save the model
with open('vgg19_1408.pb', "wb") as ofile:
    ofile.write(frozen_graph.SerializeToString())

INFO:tensorflow:Froze 26 variables.
Converted 26 variables to const ops.


In [37]:
batch_size = 1 # change to 128 when you use batch
workspace_size_bytes = 1 << 30
precision_mode = 'FP16' # use 'FP32' for K80
trt_gpu_ops = tf.GPUOptions(per_process_gpu_memory_fraction = 0.50)
print (trt_gpu_ops)

tf.reset_default_graph()
g1 = tf.Graph()
with g1.as_default():
    #
    # Create graph
    #
    in_images = tf.placeholder(tf.string, name='in_images')
    decoded_input = tf.image.decode_png(in_images, channels=3)
    float_input = tf.cast(decoded_input, dtype=tf.float32)
    # (224, 224, 3) -> (n, 224, 224, 3)
    rgb_input = tf.expand_dims(
        float_input,
        axis=0)
    # For VGG preprocess, reduce means and convert to BGR
    slice_red = tf.slice(
        rgb_input,
        [0, 0, 0, 0],
        [1, 224, 224, 1])
    slice_green = tf.slice(
        rgb_input,
        [0, 0, 0, 1],
        [1, 224, 224, 1])
    slice_blue = tf.slice(
        rgb_input,
        [0, 0, 0, 2],
        [1, 224, 224, 1])
    sub_red = tf.subtract(slice_red, 123.68)
    sub_green = tf.subtract(slice_green, 116.779)
    sub_blue = tf.subtract(slice_blue, 103.939)
    transferred_input = tf.concat(
        [sub_blue, sub_green, sub_red],
        3)
    #
    # Transform to vectors
    #
    with tf.Session(config=tf.ConfigProto(gpu_options=trt_gpu_ops)) as s1:
        with open('/content/drive/My Drive/TRT/image227.png', 'rb') as f:
            data1 = f.read()
            feed_dict = {
                in_images: data1
            }
            imglist1 = s1.run([transferred_input], feed_dict=feed_dict)
            image1 = imglist1[0]
            

per_process_gpu_memory_fraction: 0.5



# Print the TF graph

In [38]:
classifier_model_file = './vgg19_1408.pb'
classifier_graph_def = tf.GraphDef()
with tf.gfile.Open(classifier_model_file, 'rb') as f:
    data = f.read()
    classifier_graph_def.ParseFromString(data)
print('Loaded classifier graph def')
nodes = iter(classifier_graph_def.node)
print(next(nodes))

Loaded classifier graph def
name: "input_1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 224
      }
      dim {
        size: 224
      }
      dim {
        size: 3
      }
    }
  }
}



# Generate TRT graph

In [39]:
output_names = ['concat_1408/concat'] # concat_1408, TF changes teh names depending on the SESSION

trt_graph_def = trt.create_inference_graph(
    input_graph_def=classifier_graph_def,    
    outputs=output_names,
    max_batch_size=batch_size,
    max_workspace_size_bytes=workspace_size_bytes,
    precision_mode=precision_mode)
#trt_graph_def=trt.calib_graph_to_infer_graph(trt_graph_def) # For only 'INT8'
print('Generated TensorRT graph def')

Generated TensorRT graph def


# Generate Tensor

In [40]:
tf.reset_default_graph()
g2 = tf.Graph()
with g2.as_default():
    trt_x, trt_y = tf.import_graph_def(
        trt_graph_def,
        return_elements=['input_1:0', 'concat_1408/concat:0'])
        #return_elements=['input:0', 'resnet_v1_50/block1/unit_2/bottleneck_v1/conv3'])
    
    print('Generated tensor by TensorRT graph')

Generated tensor by TensorRT graph


In [42]:
from tqdm import tqdm 
    
iterations = range(10)
    
with tf.Session(graph=g2, config=tf.ConfigProto(gpu_options=trt_gpu_ops)) as s2:
    
    #
    # predict image1 (tiger)
    #
    feed_dict2 = {
        trt_x: image1
    }    
    
#     print (feed_dict)
    start_time = time.process_time()    
    result = s2.run([trt_y], feed_dict=feed_dict2) # dont count 1st graph loading time
    print ("result shape: {}".format( len(result)))
    nd_result = result[0]
    print ("nd_result shape: {}".format( nd_result.shape))
    # remove row's dimension
    onedim_result = nd_result[0,]    
    print ("onedim_result shape: {}".format( onedim_result))
    stop_time = time.process_time()        
    print('{:.2f} milliseconds'.format((stop_time-start_time)*1000))
    
    start_time = time.process_time()          
    for i in tqdm(iterations):
        result = s2.run([trt_y], feed_dict=feed_dict2)        
    stop_time = time.process_time()
        
    print('For bench= {}, {:.2f} milliseconds per op'.format(iterations, (stop_time-start_time)*1000/ (len(iterations))))
    #
    # predict image2 (lion)
    #
    %timeit s2.run([trt_y], feed_dict=feed_dict2)
    %timeit s2.run([trt_y], feed_dict=feed_dict2)
    %timeit s2.run([trt_y], feed_dict=feed_dict2)
    

 40%|████      | 4/10 [00:00<00:00, 31.72it/s]

result shape: 1
nd_result shape: (1, 1408)
onedim_result shape: [1.4889337e+02 4.3705666e+01 6.3620472e+01 ... 0.0000000e+00 1.6365470e-02
 0.0000000e+00]
202.52 milliseconds


100%|██████████| 10/10 [00:00<00:00, 32.89it/s]


For bench= range(0, 10), 34.82 milliseconds per op
10 loops, best of 3: 22.5 ms per loop
10 loops, best of 3: 22.3 ms per loop
10 loops, best of 3: 22.3 ms per loop


In [0]:
# Do we have cuda?!
!which nvcc
!nvcc --version

In [0]:
# Let's check if a GPU accelerator card is attached in our machine:
!ls -l /dev/nv*

In [0]:
# This works too, GPU count and name
!nvidia-smi -L

### Use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'

In [0]:
! nvidia-smi

In [0]:
!lscpu |grep 'Model name'

In [0]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

In [0]:
#memory that we can use
!cat /proc/meminfo | grep 'MemAvailable'

In [0]:
#hard disk that we can use
!df -h / | awk '{print $4}'

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [0]:
# This magic will create a new CU file
# To write a CUDA C program, we need to:

#Create a source code fi le with the special fi le name extension of .cu.
#Compile the program using the CUDA nvcc compiler.
#Run the executable file from the command line, which contains the kernel code executable on the GPU.

%%file version.cu
#include <thrust/version.h>
#include <iostream>

int main(void)
{
  int major = THRUST_MAJOR_VERSION;
  int minor = THRUST_MINOR_VERSION;

  std::cout << "Thrust v" << major << "." << minor << std::endl;

  return 0;
}

In [0]:
# nvcc is the CUDA compiler 
!nvcc version.cu -o version
!./version

###  Memory footprint support libraries/code

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


In [0]:
import sys
sys.version


In [0]:
# !pip3 install torch==0.4
# !pip3 install torchvision

!pip3 install 'torch==0.4.0'
!pip3 install 'torchvision==0.2.1'
!pip3 install --no-cache-dir -I 'pillow==5.1.0'

# Restart Kernel
# This workaround is needed to properly upgrade PIL on Google Colab.
import os
#os._exit(0)

## Let's print the versions

In [0]:
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

use_cuda = torch.cuda.is_available()
# use_cuda = False

print("USE CUDA=" + str (use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor